In [1]:
import pandas as pd
from utils import * 
import geopandas as gpd
from main import *

In [2]:
datapaths = {
    "Wind": "data/wind/ez_gis.plant_power_eia_v8_wind.shp",
    "GDP": "data/gdp_data.csv",
    "Solar": "data/solar_data.csv",
    "education": "data/education_data.csv",
    "private_schools": "data/private_school_data.csv",
    "DEC_race": "data/race_dec_data.csv",
    "ACS_race": "data/race_acs_data.csv",
    "election": "data/election_data.csv",
    "income": "data/income_data.csv",
    "unemployment": "data/unemployment_data.csv",
    "NREL_Electric": "data/NREL_Electric_data.csv",
    "Rural_Urban": "data/rural_urban.csv",
    'Population Data': 'data/pop_data.csv',
    "solar_roof": "data/solar_roof_data.csv"
}
bounding_box = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/county_bounding_boxes.csv", dtype={"FIPS State": str, "FIPS County": str})

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
FIPS = pd.read_csv("data/FIPS.csv", dtype={"FIPS State": str, "FIPS County": str})
FIPS

,State,County Name,FIPS State,FIPS County
0,Alabama,Autauga,01,001
1,Alabama,Baldwin,01,003
2,Alabama,Barbour,01,005
3,Alabama,Bibb,01,007
4,Alabama,Blount,01,009
...,...,...,...,...
3106,Wyoming,Sweetwater,56,037
3107,Wyoming,Teton,56,039
3108,Wyoming,Uinta,56,041
3109,Wyoming,Washakie,56,043


In [5]:
solar = get_solar(datapaths["Solar"], bounding_box, size='all')
wind = get_wind(datapaths["Wind"], bounding_box)
edu_18_24 = get_education_18_24(datapaths["education"])
edu_25 = get_education_25_over(datapaths["education"])
priv_sch = get_no_priv_schools(datapaths["private_schools"])
race_dec = get_race_dec(datapaths["DEC_race"])
race_acs = get_race_acs(datapaths["ACS_race"])
elections = get_election(datapaths["election"])
income = get_income(datapaths["income"])
unemployment = get_unemployment(datapaths["unemployment"])
electric = NREL_Electric(datapaths["NREL_Electric"])
save_datapaths = "missing_counties/"

/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/utils.py:269: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,40,41,42,43,44,46,47,48,49,50,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,107,108,109,118,119,122,123,124,125,132,133,134,135,136,137,138,139,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,162,163,164,165,168,169,170,171,174,175,176,177,180,181,182,183,186,187,188,189,192,193,194,195,222,223,224,225,228,229,230,231,234,235,236,237,238,239,240,241,242,243,244,245,258,260,261,262,263,264,265,266,267,268,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,290,291,292,293,294,296,297,298,299,300,302,303,304,305,306,308,309,310,311,312,313,314,31

In [6]:
# get the counties that are in FIPS but not in solar using State Name and County Name columns
FIPS['State_County'] = FIPS['State'] + '_' + FIPS['County Name']
solar['State_County'] = solar['State'] + '_' + solar['County Name']

# Filter for state-county pairs in FIPS but not in solar
FIPS_not_in_solar = FIPS[~FIPS['State_County'].isin(solar['State_County'])]

# Drop the combined key column if you don't need it anymore
FIPS_not_in_solar = FIPS_not_in_solar.drop(columns=['State_County'])

# Resulting DataFrame with state-county pairs only in FIPS
FIPS_not_in_solar.to_csv(save_datapaths + "solar_missing.csv", index=False)

In [36]:
FIPS_not_in_solar

,State,County Name,FIPS State,FIPS County
0,Alabama,Autauga,01,001
1,Alabama,Baldwin,01,003
2,Alabama,Barbour,01,005
3,Alabama,Bibb,01,007
4,Alabama,Blount,01,009
...,...,...,...,...
3105,Wyoming,Sublette,56,035
3107,Wyoming,Teton,56,039
3108,Wyoming,Uinta,56,041
3109,Wyoming,Washakie,56,043


In [35]:
def get_missing_counties(data, factor, base):
    base['State_County'] = base['State'] + '_' + base['County Name']
    data['State_County'] = data['State'] + '_' + data['County Name']
    
    not_in = base[~base['State_County'].isin(data['State_County'])]
    not_in = not_in.drop(columns=['State_County'])
    return not_in